In [ ]:
import os
import stocklab
import stocklab_twse.data_bundle
import stocklab_twse.analysis_bundle

config_file = 'config.yml'
stocklab.configure(config_file)

## Ad-hoc node

In [ ]:
"""Ad-hoc node template"""
from stocklab.node import *
from stocklab import DataIdentifier as DI
from stocklab_twse.utils.datetime import Date
from stocklab_twse.utils import date_range

class FooNode(Node):
    args = Args(
            stock = Arg(),
            date = Arg(type=Date),
            )

    def evaluate(stock, date):
        raise NotImplementedError()


In [ ]:
"""Installation"""
from stocklab.core import bundle
from stocklab.core.node import flush_cache
bundle.register(FooNode, allow_overwrite=True)
flush_cache()

In [ ]:
"""Usage"""
raise NotImplementedError()

## Analysis feature

In [ ]:
"""Plot stuff"""
#http://mrjbq7.github.io/ta-lib/
#https://ta-lib.org/d_api/ta_setunstableperiod.html
import numpy as np
from stocklab import DataIdentifier as DI
from stocklab_twse.utils import date_range
from stocklab_twse.utils.plot import plot

n = 160
sid = '2330'

dates = list(date_range(end=20181201, window=n))
dates.sort()
ohlcs = [DI('DailyData')(stock=sid, date=d) for d in dates]
ohlcs = [(inf['open'], inf['max'], inf['min'], inf['close'], inf['delta_price_share']) for inf in ohlcs]
ohlcs = np.asarray(ohlcs)

import talib
#talib.set_unstable_period('EMA', 70)
talib.set_unstable_period('ALL', 0)
patterns = talib.get_function_groups()['Pattern Recognition']
signs = np.array([getattr(talib, p)(ohlcs[:,0], ohlcs[:,1], ohlcs[:,2], ohlcs[:,3]) for p in patterns])
foo = talib.CDL3BLACKCROWS(ohlcs[:,0], ohlcs[:,1], ohlcs[:,2], ohlcs[:,3])
mom = talib.MOM(ohlcs[:,-2], timeperiod=5)
inphase, quadrature = talib.HT_PHASOR(ohlcs[:,-2])
#aux = {'UB': upperband, 'MB': middleband, 'LB': lowerband}
aux = {'foo': foo}
taux = {'vol': ohlcs[:,-1]}
taux = {'inphase': inphase, 'quad': quadrature}

#sgn = np.zeros(n)
#sgn = signs[9]
golden = np.array([
    DI('GoldenSign')(stock=sid, date=d, window=15, gain=1.02, mode='time_first')
    for d in dates
])
plot(dates, ohlcs[:,0:4], signs=golden, aux=aux)
#plot(dates, ohlcs[:,0:4], signs=sgn, aux=aux, top_aux=taux)
#print([(s>0).sum() for s in signs])
#print([(s<0).sum() for s in signs])

In [ ]:
"""Performance evaluation (backtracing)"""
from stocklab_twse.utils import Simulator

signs = np.array([getattr(talib, p)(ohlcs[:,0], ohlcs[:,1], ohlcs[:,2], ohlcs[:,3]) for p in patterns])
sim = Simulator()
hold = sim(dates, golden, ohlcs[:,-2])
plot(dates, ohlcs[:,0:4], signs=golden, top_aux={'sim': hold})